### Squid Games ML

https://en.wikipedia.org/wiki/Squid_Game_season_1

#### Season 1 episode 7

There are 15 2x tiles of glass each with a 0.5 probability of being a tempered glass.
- To model this each step on a single level of either a glass or a tempered glass tile is a Bernoulli distribution with probability 0.5

- To model a players death tile distribution - the tile where it will eventually fall - is a Geometric distribution - the number of Bernoulli trials until success - in our case success = death

To model each player start at the last tile where the previous player dies I use cumsum on the samples...


In [ ]:
import os

%pip install --quiet tensorflow
%pip install --quiet tf-keras
%pip install --quiet tensorflow-probability
%pip install --quiet arviz
%pip install --quiet seaborn

import numpy as np
import arviz as az
import seaborn as sns
from collections import OrderedDict
import tensorflow as tf
import tensorflow_probability as tfp

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
NO_SAMPLES = 10000
MAX_TILE = 15

player = tfp.distributions.Geometric(probs=0.5)

samples = np.clip(1 + player.sample(NO_SAMPLES).numpy(), 1, MAX_TILE+1).astype(int)


az.plot_dist(samples)

In [ ]:
NO_PLAYERS=10

players = tfp.distributions.Independent(
    tfp.distributions.Geometric(probs=[0.5]*NO_PLAYERS),
    reinterpreted_batch_ndims=1
)

samples = np.clip(1 + np.cumsum(players.sample(NO_SAMPLES).numpy(), axis=1), 1, MAX_TILE+1).astype(int)

data = [(f'Player {i+1}', samples[:, i]) for i in range(NO_PLAYERS)]


sns.displot(dict(data[:5]), kde=True)
sns.displot(dict(data[5:]), kde=True)